In [5]:
# Cell 1: Imports and Firebase Initialization
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import datetime # For Timestamps

# --- IMPORTANT SETUP ---
# Replace 'path/to/your/serviceAccountKey.json' with the actual path to your downloaded key
CRED_PATH = "G:/FIRESTOREKEYS/english-language-art-helper-firebase-adminsdk-fbsvc-e5e8256254.json" # MAKE SURE TO UPDATE THIS

try:
    # Check if the app is already initialized to prevent re-initialization error in Notebooks
    if not firebase_admin._apps:
        cred = credentials.Certificate(CRED_PATH)
        firebase_admin.initialize_app(cred)
        print("Firebase Admin SDK initialized successfully.")
    else:
        print("Firebase Admin SDK already initialized.")
except Exception as e:
    print(f"Error initializing Firebase Admin SDK: {e}")
    print("Please ensure your service account key JSON file is correctly placed and the path is updated.")
    # In a notebook, you might want to raise the exception or handle it differently
    # For now, we'll just print and potentially exit if we were in a script.
    # If this cell fails, subsequent cells will also fail.

db = firestore.client()
print("Firestore client obtained.")


Firebase Admin SDK already initialized.
Firestore client obtained.


In [ ]:
# Cell 2: Define the Sample Data Setup Function

def setup_sample_data(db_client):
    print("\nStarting to set up sample data...")
    db = db_client # Use the passed-in client

    # 1. Add a sample Level
    print("Attempting to add sample level...")
    levels_col_ref = db.collection('levels')
    level_doc_ref = levels_col_ref.document('gsat_english') # Using a specific ID
    try:
        level_doc_ref.set({
            'nameEnglish': 'General Scholastic Ability Test (English)',
            'nameTraditionalChinese': '大學學科能力測驗 (英文)',
            'shortNameEnglish': 'GSAT',
            'shortNameTraditionalChinese': '學測英文',
            'educationSystem': 'Taiwan University Entrance',
            'order': 1,
            'createdAt': firestore.SERVER_TIMESTAMP
        })
        print(f"  Level '{level_doc_ref.id}' added/updated successfully.")
    except Exception as e:
        print(f"  Error adding/updating level '{level_doc_ref.id}': {e}")

    # 2. Add a sample Vocabulary Item
    print("Attempting to add sample vocabulary item...")
    vocab_col_ref = db.collection('vocabularyItems')
    # Let Firestore auto-generate an ID for this new item
    # If you run this cell multiple times, it will add a NEW vocab item each time.
    # For idempotency with auto-IDs, you'd need to query first or use fixed IDs.
    vocab_doc_ref = vocab_col_ref.document() 
    try:
        vocab_doc_ref.set({
            'englishWord': 'ephemeral', # Changed word for variety if run multiple times
            'traditionalChinese': '短暫的',
            'partOfSpeech': 'adjective',
            'levelIds': ['gsat_english'],
            'exampleSentences': [
                {'english': 'The beauty of the cherry blossoms is ephemeral.', 'traditionalChinese': '櫻花之美是短暫的。'}
            ],
            'tags': ['time', 'nature'],
            'createdAt': firestore.SERVER_TIMESTAMP,
            'updatedAt': firestore.SERVER_TIMESTAMP
        })
        print(f"  Vocabulary item '{vocab_doc_ref.id}' added successfully.")
    except Exception as e:
        print(f"  Error adding vocabulary item '{vocab_doc_ref.id}': {e}")

    # 3. Add a sample Quiz (Grammar focus) with a Question
    print("Attempting to add sample quiz...")
    quizzes_col_ref = db.collection('quizzes')
    quiz_doc_ref = quizzes_col_ref.document('grammar_quiz_01') # Specific ID
    try:
        quiz_doc_ref.set({
            'titleEnglish': 'Basic Verb Tenses Quiz',
            'titleTraditionalChinese': '基礎動詞時態測驗',
            'levelIds': ['gsat_english'],
            'quizFocus': 'grammar',
            'createdAt': firestore.SERVER_TIMESTAMP,
            'updatedAt': firestore.SERVER_TIMESTAMP
        })
        print(f"  Quiz '{quiz_doc_ref.id}' added/updated successfully.")

        print(f"  Attempting to add question to quiz '{quiz_doc_ref.id}'...")
        quiz_questions_col_ref = quiz_doc_ref.collection('questions')
        question1_doc_ref = quiz_questions_col_ref.document('q1') # Specific ID for the question
        question1_doc_ref.set({
            'questionTextEnglish': 'She ___ to the store yesterday.',
            'questionTextTraditionalChinese': '她昨天___去商店。',
            'questionType': 'multiple_choice',
            'options': [
                {'optionId': 'A', 'textEnglish': 'go', 'textTraditionalChinese': '去'},
                {'optionId': 'B', 'textEnglish': 'goes', 'textTraditionalChinese': '去'},
                {'optionId': 'C', 'textEnglish': 'went', 'textTraditionalChinese': '去了'},
                {'optionId': 'D', 'textEnglish': 'gone', 'textTraditionalChinese': '去了'}
            ],
            'correctOptionId': 'C',
            'explanationEnglish': '"Went" is the past tense of "go".',
            'explanationTraditionalChinese': '「Went」是「go」的過去式。',
            'order': 1
        })
        print(f"    Question '{question1_doc_ref.id}' added/updated for quiz '{quiz_doc_ref.id}'.")
    except Exception as e:
        print(f"  Error adding/updating quiz '{quiz_doc_ref.id}' or its questions: {e}")

    # 4. Add a sample Article with a Comprehension Question
    print("Attempting to add sample article...")
    articles_col_ref = db.collection('articles')
    article_doc_ref = articles_col_ref.document('sustainability_article_01') # New specific ID
    try:
        article_doc_ref.set({
            'title': 'The Importance of Sustainable Practices',
            'sourceUrl': 'http://example.com/sustainability',
            'content': 'Sustainable practices are crucial for the future of our planet...',
            'levelIds': ['gsat_english'],
            'tags': ['environment', 'sustainability'],
            'hasComprehensionQuestions': True,
            'scrapedAt': firestore.SERVER_TIMESTAMP,
            'createdAt': firestore.SERVER_TIMESTAMP,
            'updatedAt': firestore.SERVER_TIMESTAMP
        })
        print(f"  Article '{article_doc_ref.id}' added/updated successfully.")

        print(f"  Attempting to add comprehension question to article '{article_doc_ref.id}'...")
        article_questions_col_ref = article_doc_ref.collection('questions')
        article_q1_ref = article_questions_col_ref.document('aq1') # Specific ID
        article_q1_ref.set({
            'questionTextEnglish': 'What is a key reason for sustainable practices mentioned?',
            'questionTextTraditionalChinese': '文中提到永續實踐的一個關鍵原因是什麼？',
            'questionType': 'short_answer',
            'correctAnswer': {'text': 'future of our planet'},
            'explanationEnglish': 'The article emphasizes planetary health.',
            'explanationTraditionalChinese': '文章強調地球健康。',
            'order': 1
        })
        print(f"    Question '{article_q1_ref.id}' added/updated for article '{article_doc_ref.id}'.")
    except Exception as e:
        print(f"  Error adding/updating article '{article_doc_ref.id}' or its questions: {e}")

    print("\nSample data setup function finished execution.")



In [ ]:
# Cell 3: Execute the Setup (with confirmation)

# Make sure the 'db' variable from Cell 1 is available
if 'db' in locals() or 'db' in globals():
    proceed = input("This script will add/update sample data in Firestore. Proceed? (yes/no): ")
    if proceed.lower() == 'yes':
        setup_sample_data(db) # Pass the db client to the function
        print("\nProcess complete. Check your Firestore console.")
    else:
        print("Operation cancelled by user.")
else:
    print("Firestore client 'db' not found. Please ensure Cell 1 executed successfully.")

In [8]:
# Cell 4 (or a new cell): Update specific article questions

if 'db' not in globals() and 'db' not in locals():
    print("Firestore client 'db' not found. Please ensure Cell 1 (Firebase Initialization) was executed successfully.")
else:
    print("Attempting to update questions for article 'sustainability_article_01'...")
    try:
        article_doc_ref = db.collection('articles').document('sustainability_article_01')
        
        # Check if the article exists before trying to add questions to its subcollection
        article_snapshot = article_doc_ref.get()
        if not article_snapshot.exists:
            print(f"  Error: Article 'sustainability_article_01' does not exist. Please run the setup cell first or ensure the ID is correct.")
        else:
            print(f"  Article 'sustainability_article_01' found. Proceeding with question updates.")
            article_questions_col_ref = article_doc_ref.collection('questions')

            # 1. Modify/Ensure aq1 (short_answer question)
            # This will overwrite the existing aq1 or create it if it's missing.
            print("  Updating/Creating question 'aq1' (short_answer)...")
            aq1_ref = article_questions_col_ref.document('aq1')
            aq1_ref.set({
                'questionTextEnglish': 'What is a key reason for sustainable practices mentioned?',
                'questionTextTraditionalChinese': '文中提到永續實踐的一個關鍵原因是什麼？',
                'questionType': 'short_answer',
                'correctAnswer': {'text': 'future of our planet'}, # Ensure this structure
                'explanationEnglish': 'The article emphasizes planetary health.',
                'explanationTraditionalChinese': '文章強調地球健康。',
                'order': 1,
                'points': 5  # Added points for consistency
            })
            print(f"    Question 'aq1' processed successfully.")

            # 2. Add aq2 (multiple_choice question)
            # This will create aq2 or overwrite it if it somehow already exists with that ID.
            print("  Adding/Updating question 'aq2' (multiple_choice)...")
            aq2_ref = article_questions_col_ref.document('aq2')
            aq2_ref.set({
                'questionTextEnglish': 'Which of these is highlighted as a sustainable energy source in the article?',
                'questionTextTraditionalChinese': '文中強調下列哪一項為永續能源？',
                'questionType': 'multiple_choice',
                'options': [
                    {'optionId': 'A', 'textEnglish': 'Coal', 'textTraditionalChinese': '煤炭'},
                    {'optionId': 'B', 'textEnglish': 'Solar Power', 'textTraditionalChinese': '太陽能'},
                    {'optionId': 'C', 'textEnglish': 'Natural Gas', 'textTraditionalChinese': '天然氣'}
                ],
                'correctAnswer': {'optionId': 'B'}, # Correct answer references an optionId
                'explanationEnglish': 'Solar power is a renewable and sustainable energy source mentioned.',
                'explanationTraditionalChinese': '太陽能是文中提到的一種可再生且永續的能源。',
                'order': 2,
                'points': 10
            })
            print(f"    Question 'aq2' processed successfully.")
            
            # Optionally, ensure the article's 'hasComprehensionQuestions' flag is true
            article_doc_ref.update({'hasComprehensionQuestions': True, 'updatedAt': firestore.SERVER_TIMESTAMP})
            print("  Ensured 'hasComprehensionQuestions' is true for the article.")

        print("\nSpecific article question update finished.")

    except Exception as e:
        print(f"  An error occurred: {e}")



Attempting to update questions for article 'sustainability_article_01'...
  Article 'sustainability_article_01' found. Proceeding with question updates.
  Updating/Creating question 'aq1' (short_answer)...
    Question 'aq1' processed successfully.
  Adding/Updating question 'aq2' (multiple_choice)...
    Question 'aq2' processed successfully.
  Ensured 'hasComprehensionQuestions' is true for the article.

Specific article question update finished.
